In [1]:
import fitz
from PIL import Image
import wget
import os
from pdf2image import convert_from_path

# this will install poppler file
zip_url = "https://github.com/oschwartz10612/poppler-windows/releases/download/v24.02.0-0/Release-24.02.0-0.zip"
zip_filename = wget.download(zip_url)

# unzips poppler file
!unzip -o {zip_filename}

doc = fitz.open("table.pdf")
page = doc[0]
paths = page.get_drawings()  # extract existing drawings
# this is a list of "paths", which can directly be drawn again using Shape
# -------------------------------------------------------------------------
# define some output page with the same dimensions
outpdf = fitz.open()
outpage = outpdf.new_page(width=page.rect.width, height=page.rect.height)
shape = outpage.new_shape()  # make a drawing canvas for the output page
# --------------------------------------
# loop through the paths and draw them
# --------------------------------------
for path in paths:
    # ------------------------------------
    # draw each entry of the 'items' list
    # ------------------------------------
    for item in path["items"]:  # these are the draw commands
        if item[0] == "l":  # line
            shape.draw_line(item[1], item[2])
        elif item[0] == "re":  # rectangle
            shape.draw_rect(item[1])
        elif item[0] == "qu":  # quad
            shape.draw_quad(item[1])
        elif item[0] == "c":  # curve
            shape.draw_bezier(item[1], item[2], item[3], item[4])
        else:
            raise ValueError("unhandled drawing", item)
    # ------------------------------------------------------
    # all items are drawn, now apply the common properties
    # to finish the path
    # ------------------------------------------------------
    shape.finish()
shape.commit()
outpdf.save("test_file_ext2.pdf")
pdf_path = 'test_file_ext2.pdf'
doc = fitz.open(pdf_path)

# Get the poppler executable path dynamically
!which poppler > poppler_path.txt
with open('poppler_path.txt', 'r') as f:
    poppler_path = f.read().strip()

# Use the dynamic poppler path
images = convert_from_path(pdf_path, poppler_path=poppler_path)

# Save each image as a JPEG file
for i, image in enumerate(images):
    image.save(f'extracted_image{i+1}.jpg', 'JPEG')